In [451]:
"""
Скрипт для создания сводный таблиц по отчету
"""

'\nСкрипт для создания сводный таблиц по отчету\n'

In [452]:
import pandas as pd
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.utils import get_column_letter
import numpy as np
import re

In [453]:
def foo(value):
    d = (value.tolist())
    return d
    


In [454]:
str_column ='20,22' # колонки для сводной таблицы
#str_column = 'fgg'
str_target_column = '6,15' # целевая колонка

In [455]:
file_data = 'data/Сводная таблица/data.xlsx'
sheet_name = 'Лист1'
end_folder = 'data/Сводная таблица/result'

In [456]:
df = pd.read_excel(file_data,sheet_name=sheet_name)

In [457]:
# обрабатываем список колонок по которым нужно группировать
_lst_cols = re.findall(r'\d+',str_column) # находим цифры
if _lst_cols:
    lst_number_cols =  list(map(int,_lst_cols))
else:
    print('Введите порядковые номера колонок в виде цифр')

In [458]:
lst_number_cols

[20, 22]

In [459]:
_target_col =re.findall(r'\d+',str_target_column)
if _target_col:
    lst_target_number_column =  list(map(int,_target_col))
else:
    print('Введите порядковые номера колонок в виде цифр')

In [460]:
lst_target_number_column

[6, 15]

In [461]:
lst_name_column = [] # список для хранения названий колонок сводной таблицы
for value in lst_number_cols:
    if value != 0:
        lst_name_column.append(df.columns[value-1])
        

In [462]:
lst_name_column

['Образование', 'Группа']

In [463]:
lst_target_name_column = [] # список для хранения названий колонок сводной таблицы
for value in lst_target_number_column:
    if value != 0:
        lst_target_name_column.append(df.columns[value-1])

In [464]:
lst_target_name_column

['Средний балл', 'ИНН']

In [465]:

dct_func = {'Сумма':'sum','Среднее':'mean','Медиана':'median','Минимум':'min','Максимум':'max','Количество':'count'}

In [466]:
wb = openpyxl.Workbook() # создаем объект
for idx,items in enumerate(dct_func.items()):
    name_sheet,name_func = items # распаковываем
    wb.create_sheet(title=name_sheet,index=idx) # создаем лист
    
    df[lst_target_name_column] = df[lst_target_name_column].applymap(lambda x: x if isinstance(x,(int,float)) else 0)
    
    temp_df =pd.pivot_table(df,
              index=lst_name_column,
              values=lst_target_name_column,             
              aggfunc=name_func,
                           fill_value=0)
    temp_df = temp_df.reset_index()

    
    for row in dataframe_to_rows(temp_df,index=False,header=True):
        wb[name_sheet].append(row)
    for column in wb[name_sheet].columns:
        max_length = 0
        column_name = get_column_letter(column[0].column)
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        wb[name_sheet].column_dimensions[column_name].width = adjusted_width
        
del wb['Sheet']
    



In [467]:
wb.sheetnames

['Сумма', 'Среднее', 'Медиана', 'Минимум', 'Максимум', 'Количество']

In [468]:
wb.save(f'{end_folder}/Тест.xlsx')

In [469]:
pd.pivot_table(df,
              index=lst_name_column,
              values=['Фамилия','Направление_подготовки'],             
              aggfunc='count',
                           fill_value=0)

Направление_подготовки  Фамилия
Образование              Группа                                 
Основное общее           АГ-596                      20       20
                         АД-393                      20       20
                         АЛ-480                      20       20
                         АП-380                      20       20
                         АЯ-322                      20       20
                         АЯ-849                      20       20
                         БВ-523                      20       20
                         БЕ-281                      20       20
                         БЕ-944                      20       20
                         БН-321                      20       20
                         БЯ-877                      20       20
                         БЯ-928                      20       20
                         ВД-204                      20       20
                         ВЕ-129                      20       20
                         ГВ-833                      20       20
                         ГГ-759                      20       20
                         ДА-989                      15       15
                         ДВ-244                      20       20
                         ДО-951                      20       20
                         ДП-786                      20       20
                         ЕА-479                      20       20
                         ЕД-195                      20       20
                         ЕЕ-374                      20       20
                         ЕП-951                      20       20
                         КА-426                      20       20
                         КА-733                      20       20
                         КВ-838                      20       20
                         КД-856                      20       20
                         КО-589                      20       20
                         ЛЛ-514                      20       20
                         ЛН-300                      20       20
                         ЛО-368                      20       20
                         ЛП-187                      20       20
                         НБ-368                      20       20
                         НЕ-643                      20       20
                         НО-608                      20       20
                         НП-652                      20       20
                         НЭ-787                      20       20
                         НЯ-427                      20       20
                         ОБ-280                      20       20
                         ОВ-173                      20       20
                         ОГ-541                      20       20
                         ПН-298                      20       20
                         ПН-962                      20       20
                         ПЭ-344                      19       19
                         ЭД-420                      20       20
                         ЭН-535                      20       20
                         ЯА-476                      20       20
                         ЯК-419                      20       20
                         ЯЯ-331                      20       20
Среднее общее            ДА-989                       4        4
Среднее профессиональное ДА-989                       1        1
                         ПЭ-344                       1        1